In [1]:
import numpy as np
import pandas as pd
from scipy.optimize import linprog

from mis_utils import *

In [2]:
# Definiciones de constantes

node_names = np.array(('s', '2', '3', '4', '5', 't'))

# Balances: Sale un producto desde s y llega hasta t
beq = np.array([1, 0, 0, 0, 0, -1])

NN = np.array([[0, 1, 1, 0, 0, 0],
               [0, 0, 0, 1, 0, 1],
               [0, 0, 0, 0, 1, 0],
               [0, 0, 0, 0, 0, 1],
               [0, 0, 0, 0, 0, 1],
               [0, 0, 0, 0, 0, 0]])

# Matrices resultantes de NN a NA
Aeq, arc_idxs = nn2na(NN, node_names = node_names, show_results = True)

# Guardo los nombres de los nodo-arco-nodo posibles
nan_names = get_col_names(NN, node_names, as_numpy=True, sep = "->")

# Restricciones, l <= x <= u
# Entre 0 y 1 sería lo teóricamente correcto, aunque como hay un límite que como el modelo indica, es de un producto
min_bound = 0
max_bound = 1
bounds = tuple([(min_bound, max_bound) for arcs in range(0, Aeq.shape[1])])

[[0 1]
 [0 2]
 [1 3]
 [1 5]
 [2 4]
 [3 5]
 [4 5]]
Input: 
 [[0 1 1 0 0 0]
 [0 0 0 1 0 1]
 [0 0 0 0 1 0]
 [0 0 0 0 0 1]
 [0 0 0 0 0 1]
 [0 0 0 0 0 0]]

Column names: s2-s3-24-2t-35-4t-5t

Output: 
 [[ 1  1  0  0  0  0  0]
 [-1  0  1  1  0  0  0]
 [ 0 -1  0  0  1  0  0]
 [ 0  0 -1  0  0  1  0]
 [ 0  0  0  0 -1  0  1]
 [ 0  0  0 -1  0 -1 -1]]



In [3]:
# Vector de costos
# Con el orden de los nombres de columnas, sacado de los resultados. Column names: s2-s3-24-2t-35-4t-5t
C = np.array([2, 1, 2, 5, 2, 1, 2])

In [4]:
pd.DataFrame(data=NN, columns=node_names, index=node_names)

,s,2,3,4,5,t
s,0,1,1,0,0,0
2,0,0,0,1,0,1
3,0,0,0,0,1,0
4,0,0,0,0,0,1
5,0,0,0,0,0,1
t,0,0,0,0,0,0


In [5]:
pd.DataFrame(data=Aeq, columns=nan_names, index=node_names)

,s->2,s->3,2->4,2->t,3->5,4->t,5->t
s,1,1,0,0,0,0,0
2,-1,0,1,1,0,0,0
3,0,-1,0,0,1,0,0
4,0,0,-1,0,0,1,0
5,0,0,0,0,-1,0,1
t,0,0,0,-1,0,-1,-1


In [6]:
resume_df = pd.DataFrame(bounds, index=nan_names, columns=['Min bound', 'Max bound'])
resume_df['Costs'] = C

In [7]:
resume_df

,Min bound,Max bound,Costs
s->2,0,1,2
s->3,0,1,1
2->4,0,1,2
2->t,0,1,5
3->5,0,1,2
4->t,0,1,1
5->t,0,1,2


In [8]:
# Resumen
print('## Optimizer inputs ## \n\n'
      'Cost vector: %s \n\n'
      'Columns: %s \n\n'
      'A_eq Node-Arc matrix:\n%s \n\n'
      'b_eq demand-supply vector: %s \n\n'
      'Bounds of each X arc variable: %s \n' % (C, nan_names, Aeq, beq, bounds))

## Optimizer inputs ## 

Cost vector: [2 1 2 5 2 1 2] 

Columns: ['s->2' 's->3' '2->4' '2->t' '3->5' '4->t' '5->t'] 

A_eq Node-Arc matrix:
[[ 1  1  0  0  0  0  0]
 [-1  0  1  1  0  0  0]
 [ 0 -1  0  0  1  0  0]
 [ 0  0 -1  0  0  1  0]
 [ 0  0  0  0 -1  0  1]
 [ 0  0  0 -1  0 -1 -1]] 

b_eq demand-supply vector: [ 1  0  0  0  0 -1] 

Bounds of each X arc variable: ((0, 1), (0, 1), (0, 1), (0, 1), (0, 1), (0, 1), (0, 1)) 



# Resuelvo con el método por defecto (interior-point)

In [9]:
# Optimización
res = linprog(C, A_eq=Aeq, b_eq=beq, bounds=bounds, method='interior-point')

selarcs = get_selected_arcs(res.x.round().astype(int), nan_names)

/home/diego/.virtualenvs/OS11/lib/python3.6/site-packages/ipykernel_launcher.py:2: OptimizeWarning: A_eq does not appear to be of full row rank. To improve performance, check the problem formulation for redundant equality constraints.
  


In [10]:
print('## Results ##')
print('The raw solution will be: %s' % res.x.round().astype(int))
print('The arcs that make the shortest path will be (from, to): %s' % selarcs)
print('The minimum cost will be: %0.2f ' % res.fun)

## Results ##
The raw solution will be: [1 0 1 0 0 1 0]
The arcs that make the shortest path will be (from, to): ['s->2', '2->4', '4->t']
The minimum cost will be: 5.00 


# Resuelvo con el método simplex

In [11]:
# Optimización
res_simplex = linprog(C, A_eq=Aeq, b_eq=beq, bounds=bounds, method='simplex')

selarcs = get_selected_arcs(res_simplex.x.round().astype(int), nan_names)

/home/diego/.virtualenvs/OS11/lib/python3.6/site-packages/ipykernel_launcher.py:2: OptimizeWarning: A_eq does not appear to be of full row rank. To improve performance, check the problem formulation for redundant equality constraints.
  


In [12]:
res.x

array([5.01717767e-01, 4.98282233e-01, 5.01717767e-01, 5.19045381e-11,
       4.98282233e-01, 5.01717767e-01, 4.98282233e-01])

In [13]:
res_simplex.x

array([0., 1., 0., 0., 1., 0., 1.])

In [14]:
print('## Results ##')
print('The raw solution will be: %s' % res_simplex.x.round().astype(int))
print('The arcs that make the shortest path will be (from, to): %s' % selarcs)
print('The minimum cost will be: %0.2f ' % res_simplex.fun)

## Results ##
The raw solution will be: [0 1 0 0 1 0 1]
The arcs that make the shortest path will be (from, to): ['s->3', '3->5', '5->t']
The minimum cost will be: 5.00 


# En este caso, el problema tiene múltiples soluciones, habiendo varias soluciones, cada algoritmo puede elegir una u otra (y lo hace)

# El método simplex no resuelve problemas enteros, que hay respecto a eso?

In [15]:
resume_df['Solution inner_point'] = res.x.round().astype(int)

In [16]:
resume_df['Solution Simplex'] = res_simplex.x.round().astype(int)

In [17]:
resume_df

,Min bound,Max bound,Costs,Solution inner_point,Solution Simplex
s->2,0,1,2,1,0
s->3,0,1,1,0,1
2->4,0,1,2,1,0
2->t,0,1,5,0,0
3->5,0,1,2,0,1
4->t,0,1,1,1,0
5->t,0,1,2,0,1


In [18]:
if (resume_df['Solution Simplex']!=resume_df['Solution inner_point']).sum()>0:
    print("Las soluciones son diferentes")
else:
    print("Las soluciones son iguales")

if round(res_simplex.fun) == round(res.fun):
    print("Los costos dan iguales: {}".format(res_simplex.fun))
    
else:
    print("Los costos son diferentes")

Las soluciones son diferentes
Los costos dan iguales: 5.0


In [19]:
print('The arcs that make the shortest path trough simplex will be (from, to): %s' % selarcs)

The arcs that make the shortest path trough simplex will be (from, to): ['s->3', '3->5', '5->t']


In [20]:
print('The arcs that make the shortest path trough inner-point will be (from, to): %s' % selarcs)

The arcs that make the shortest path trough inner-point will be (from, to): ['s->3', '3->5', '5->t']


In [21]:
print("El error para el método inner: ", np.sum(np.abs(res.x.round() - res.x)))

El error para el método inner:  2.9896934005812197


In [22]:
np.sum(np.abs(res_simplex.x.round() - res_simplex.x))

0.0

In [23]:
print("Hay que tener cuidado con la sensibilidad")

Hay que tener cuidado con la sensibilidad
